In [2]:
%load_ext autoreload
%autoreload 2

import math
import argparse
import h5py
import importlib
import numpy as np
import torch
import time
import sys
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_

from models.vae_flow import *
from models.flow import add_spectral_norm, spectral_norm_power_iteration
from configs import Configs
from utils.plotting import get_projections, get_plots, MAP, offset, layer_bottom_pos, cell_thickness, Xmax, Xmin, Zmax, Zmin, get_features
from tqdm import tqdm

import k_diffusion as K


cfg = Configs()

print(cfg.__dict__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'name': 'CD_', 'comet_project': 'calo-consistency', 'Acomment': 'long baseline with lat_dim = 0, max_iter 1M, lr=1e-4 fixed, num_steps=18, bs=256, simga_max=80, epoch=2M, EMA', 'log_comet': True, 'model_name': 'epicVAE_nFlow_kDiffusion', 'latent_dim': 0, 'beta_1': 0.0001, 'beta_T': 0.02, 'sched_mode': 'quardatic', 'flexibility': 0.0, 'truncate_std': 2.0, 'latent_flow_depth': 14, 'latent_flow_hidden_dim': 256, 'num_samples': 4, 'features': 4, 'sample_num_points': 2048, 'kl_weight': 0.001, 'residual': False, 'cond_features': 2, 'norm_cond': True, 'kld_min': 1.0, 'use_epic': False, 'epic_layers': 5, 'hid_d': 128, 'sum_scale': 0.001, 'weight_norm': True, 'flow_model': 'PiecewiseRationalQuadraticCouplingTransform', 'flow_transforms': 10, 'flow_layers': 2, 'flow_hidden_dims': 128, 'tails': 'linear', 'tail_bound': 10, 'dataset': 'x36_grid', 'dataset_path': '/beegfs/desy/user/akorol/data/calo-clouds/hdf5/h

In [3]:
# path = '/beegfs/desy/user/akorol/data/calo-clouds/hdf5/high_granular_grid/validation/10-90GeV_x36_grid_regular.hdf5'
path = '/beegfs/desy/user/akorol/data/calo-clouds/hdf5/high_granular_grid/validation/50GeV_x36_grid_regular_2k_Z4.hdf5'
real_showers = h5py.File(path, 'r')['events'][:]
print(real_showers.shape)

(2000, 4, 6000)


In [157]:
real_events = get_projections(real_showers[0:2000], MAP, layer_bottom_pos)
len(real_events)
len(real_events[0])

100%|██████████| 2000/2000 [00:05<00:00, 369.44it/s]


30

In [180]:
len(real_events)

2000

In [184]:
st = time.time()
e_radial, occ_list, e_sum_list, hits_list, e_layers_list, occ_layers_list, e_layers_distibution = get_features(real_events[0:5], thr=0.05)
print(time.time()-st)

100%|██████████| 5/5 [00:00<00:00, 400.72it/s]

0.014760732650756836


In [125]:
occ_layers_list.shape

(200, 30)

In [126]:
e_layers_distibution.shape

(200, 30)

# caluclate percentile edges for layer wise occupancy

In [100]:
occ_layers_list.shape

(200, 30)

In [118]:
occ_layers_list_sum = occ_layers_list.sum(axis=0)

bin_edges = []
total = occ_layers_list_sum.sum()
running_sum = 0
fraction_interval_min = 0.0
fraction_interval_max = 0.1
for i in range(len(occ_layers_list_sum)):
    running_sum += occ_layers_list_sum[i]
    running_fraction = running_sum / total
    if (running_fraction >= fraction_interval_min):# and (running_fraction <= fraction_interval_max):
        fraction_interval_min += 0.1
        fraction_interval_max += 0.1
        bin_edges.append(i)
print(bin_edges)
print(len(bin_edges))


[0, 12, 15, 16, 18, 19, 21, 22, 24, 26, 29]
11


# binned occupancy

In [113]:
binned_occ_layer_list = []
for i in range(len(bin_edges)-1):
    occ_sub = occ_layers_list[:,bin_edges[i]:bin_edges[i+1]]
    if bin_edges[i+1] == bin_edges[-1]: # last bin
        occ_sub = occ_layers_list[:,bin_edges[i]:bin_edges[i+1]+1] # include last bin
    occ_sub = occ_sub.reshape(occ_sub.shape[0], -1)
    binned_occ_layer_list.append(occ_sub.sum(axis=1))
binned_occ_layer_list = np.array(binned_occ_layer_list)


# binned e layers

In [127]:
binned_e_layer_list = []
for i in range(len(bin_edges)-1):
    e_sub = e_layers_distibution[:,bin_edges[i]:bin_edges[i+1]]
    if bin_edges[i+1] == bin_edges[-1]: # last bin
        e_sub = e_layers_distibution[:,bin_edges[i]:bin_edges[i+1]+1] # include last bin
    e_sub = e_sub.reshape(e_sub.shape[0], -1)
    binned_e_layer_list.append(e_sub.sum(axis=1))
binned_e_layer_list = np.array(binned_e_layer_list)

print(binned_occ_layer_list.shape)

(10, 200)


# percentile edges for raidal occpuancy

In [62]:
occ_radial = e_radial[0]
occ_radial = np.sort(occ_radial)
print(occ_radial.shape)
print(occ_radial)

(139768,)
[  1.34100012   1.34100012   1.34100012 ... 242.92655452 246.69496695
 250.32959425]


In [63]:
np.unique(occ_radial)

array([  1.34100012,   1.34242459,   1.39231017, ..., 242.92655452,
       246.69496695, 250.32959425])

In [66]:
radial_edges = []
total = occ_radial.sum()
# running_sum = 0
fraction_interval_min = 0.0
fraction_interval_max = 0.1
for radius in np.arange(0,300,1e-3):
    running_sum = occ_radial[occ_radial < radius].sum()
    running_fraction = running_sum / total
    if (running_fraction > fraction_interval_min) and (running_fraction <= fraction_interval_max):
        fraction_interval_min += 0.1
        fraction_interval_max += 0.1
        radial_edges.append(radius)
        print(radius)
radial_edges[0] = 0  # set the first bin to be the minimum radius
radial_edges[-1] = 300  # set the last bin to be the maximum radius
print(radial_edges)
print(len(radial_edges))

1.342
10.717
14.862
19.209
24.325
30.299
38.206
49.13
65.822
93.888
250.33
[0, 10.717, 14.862, 19.209, 24.325, 30.299, 38.206, 49.13, 65.822, 93.888, 300]
11


# binned occ radial (for testing)

In [155]:
binned_occ_radial = []
for i in range(len(radial_edges)-1):
    mask =  (occ_radial >= radial_edges[i]) & (occ_radial < radial_edges[i+1])
    radial_sub = occ_radial[mask]
    binned_occ_radial.append(radial_sub.sum())
binned_occ_radial = np.array(binned_occ_radial)

print(binned_occ_radial.shape)
print(binned_occ_radial / binned_occ_radial.sum())

(10,)
[0.10062385 0.10010828 0.0994892  0.10000022 0.09992604 0.09989193
 0.09996292 0.10005995 0.09995954 0.09997807]


# binned e radial distributions

In [156]:
binned_e_radial = []
for i in range(len(radial_edges)-1):
    mask = (e_radial[0] >= radial_edges[i]) & (e_radial[0] < radial_edges[i+1])
    radial_sub = e_radial[1][mask]
    binned_e_radial.append(radial_sub.sum())
binned_e_radial = np.array(binned_e_radial)

print(binned_e_radial.shape)
print(binned_e_radial)

(10,)
[139614.18881015  15015.73282207   8653.53054378   5690.5348139
   4214.67461499   3128.19099195   2318.83462537   1748.81604599
   1258.42662651    768.01652786]
